# 0. 들어가며

  Kaggle 대회들의 상위 솔루션들을 살펴보면 여러 모델을 만든 뒤에 각 모델의 결과를 조합하여 하나의 결과로 만드는 방식으로 성능을 올리는 모습을 자주 볼 수 있습니다. 본 노트북은 [Mechanisms of Actions Prediction](https://www.kaggle.com/c/lish-moa/overview)이라는 대회에서 [4등을 한 팀의 솔루션](https://www.kaggle.com/kento1993/nn-svm-tabnet-xgb-with-pca-cnn-stacking-without-pp)을 리뷰하는 과정에서 알게된 **Blending**이라는 방법을 살펴봅니다.
  
  이 노트북을 보는 분들이 저와 같은 의문을 가지고 그 답을 찾기위한 검색을 하는데 시간을 쓰지 않으시기를 바라는 마음에서 요약을 하기보다는 공부를 해나간 순서대로 전부 기록하겠습니다.

# 1. MoA 및 Blending 간단 소개

  MoA의 목표는 Multi-label classification입니다. 따라서 모든 클래스에 대한 binary classification을 진행하게 됩니다. 각 클래스에 대한 모델의 아웃풋은 0과 1 사이의 실수가 됩니다.
  
  이와 같은 태스크 아래에서 여러 모델의 아웃풋을 조합하는 방법으로 가장 먼저 생각할 수 있는 것은 단순 평균입니다. 여기서 조금 더 나아가서 각 모델의 아웃풋에 가중치를 부여해서 가중 평균을 구할 수도 있을 것입니다. 이러한 관점에서 볼 때 Blending은 **어떻게 좋은 가중치를 찾을 것인가?**와 관련되어 있습니다.
  
  Blending에 대해 조금 더 자세히 들여다보고 싶으신 분은 [링크](https://mlwave.com/kaggle-ensembling-guide/)를 참고하시길 바랍니다.

In [1]:
# 필요한 라이브러리 임포트

import numpy as np
from scipy.optimize import minimize
from tqdm.auto import tqdm

  예시를 위해 True Label과 OOF Predictions을 '의도적으로' 만들겠습니다.
  
  * sample은 총 1000개, class는 총 200개로, true label은 50%의 확률로 1 또는 0입니다.
  * 모델은 총 4개가 있으며, 
    1. 모델1은 모든 class를 0으로
    2. 모델2는 0~149번째 class는 1, 나머지는 0으로
    3. 모델3은 50~199번째 class는 1, 나머지는 0으로
    4. 모델4는 50~149번째 class는 1, 나머지는 0으로
  예측하고 있습니다.

In [2]:
# Let's generate some labels and 4 prediction sets.
y = (np.random.rand(1000, 200) > 0.5).astype(int)
pred1 = np.zeros((1000, 200))
pred2 = np.zeros((1000, 200))
pred2[:, :150] = 1
pred3 = np.zeros((1000, 200))
pred3[:, 50:] = 1
pred4 = np.zeros((1000, 200))
pred4[:, 50:150] = 1

In [3]:
preds = np.array([pred1, pred2, pred3, pred4])

  Blending을 하는 코드를 살펴보기 전에 어떤 예측이 위의 True Label에 대해 손실 함수를 최소화 하는지 잠깐 생각해보겠습니다.
  
  각 class에 대한 binary classification을 하고 있기 때문에 손실 함수는 각 class에 대한 binary cross-entropy가 될 것입니다. 여기서 True Label은 50%의 확률로 1 또는 0이기 때문에 **모든 예측이 0.5가 된다면 손실 함수가 최소화** 될 것입니다.

---

MoA 4등 팀의 솔루션 중 Blending을 하는 부분의 코드는 아래와 같습니다.

```python
def get_score(weights, train_idx, oofs, labels):
        blend = np.zeros_like(oofs[0][train_idx, :])
        
        for oof, weight in zip(oofs[:-1], weights):
            blend += weight * oof[train_idx, :]
            
        blend += (1 - np.sum(weights)) * oofs[-1][train_idx, :]
        return logloss_for_multilabel(labels[train_idx, :], blend)

def get_best_weights(oofs, labels):
    weight_list = []
    weights = np.array([1/len(oofs) for x in range(len(oofs) - 1)])
    
    for n_splits in tqdm([5, 6]):
        for i in range(2):
            kf = KFold(n_splits=n_splits, random_state=i, shuffle=True)
            for fold, (train_idx, valid_idx) in enumerate(kf.split(X=oofs[0])):
                res = minimize(get_score, weights, args=(train_idx, oofs, labels), method="Nelder-Mead", tol=1e-6)
                logger.info(f"i: {i} fold: {fold} res.x: {res.x}")
                weight_list.append(res.x)
                
    mean_weight = np.mean(weight_list, axis=0)
    print(f"optimized weight: {mean_weight}")
    return mean_weight

best_weights = get_best_weights(stage3_oofs, train_features_df[stage_1_2_target_cols].values)
```

예시를 간단하게 만들기 위해 `n_splits`와 `KFold`를 제거 하겠습니다. 또한 손실 함수가 필요하기 때문에 정의하겠습니다.

In [4]:
def log_loss_numpy(y_pred, y_true):
    y_true_ravel = np.asarray(y_true).ravel()
    y_pred = np.asarray(y_pred).ravel()
    y_pred = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = np.where(y_true_ravel == 1, - np.log(y_pred), - np.log(1 - y_pred))
    return loss.mean()

* `log_loss_numpy`는 multi-label classification의 손실 함수 입니다.

In [5]:
def get_score(weights, oofs, labels):
        blend = np.zeros_like(oofs[0])

        for oof, weight in zip(oofs[:-1], weights):
            blend += weight * oof

        blend += (1 - np.sum(weights)) * oofs[-1]
        loss = log_loss_numpy(blend, labels)
        return loss

* `get_score`는 `oofs`와 `weights`를 가중합 한 뒤에 그 결과를 True label과 함께 손실 함수에 넣어 계산합니다.
<br>

* 가중합을 계산할 때 마지막 oof, `oofs[-1]`의 가중치는 `1 - np.sum(weights)`가 됩니다.
<br>

* 그에 따라 `weights의 길이`는 `oofs의 길이 - 1`이 됩니다. (`len(weights) == len(oofs)-1`)
<br>

* `get_score`는 `x0`와 함께 밑에 정의 될 `minimize` 함수의 인자로 주어지고 있습니다.

In [6]:
def get_best_weights(oofs, labels):
    x0 = np.array([1/len(oofs) for x in range(len(oofs) - 1)])
    res = minimize(get_score, x0, args=(oofs, labels, ), method="Nelder-Mead", tol=1e-10)
    print(res)
    return res.x

* `get_best_weights`는 OOF predictions의 리스트인 `oofs` 인자와 True label인 `labels` 인자를 받아서 최적의 가중치를 구하는 함수입니다. 
<br>

* scipy.optimize에 있는 `minimize`라는 함수를 이용하고 있으며, Nelder-Mead라는 방법을 이용합니다.

`minimize`에 대한 자세한 내용은 [링크](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize)를 참고하시길 바랍니다.

# 2. Nelder-Mead

  Nelder-Mead는 휴리스틱한 최적화 방법으로 gradient가 필요하지 않습니다. 그리고 **constraint를 사용할 수 없습니다**.
  
references:
1. https://sudonull.com/post/69185-Nelder-Mead-optimization-method-Python-implementation-example
2. https://codesachin.wordpress.com/2016/01/16/nelder-mead-optimization/
3. https://people.duke.edu/~ccc14/sta-663/BlackBoxOptimization.html
4. https://github.com/fchollet/nelder-mead/blob/master/nelder_mead.py

Nelder-Mead는 constraint를 사용할 수 없는데, 가중 평균의 가중치들은 constraint를 갖고 있습니다.

1. 모든 가중치들은 0과 1 사이의 실수이며,
2. 모든 가중치들의 합은 1이 되어야합니다.

MoA 4등 솔루션은 간접적으로 위와 같은 constraint를 적용하려고 시도 하였습니다.

```python
def get_score(weights, oofs, labels):
        blend = np.zeros_like(oofs[0])

        for oof, weight in zip(oofs[:-1], weights):
            blend += weight * oof

        blend += (1 - np.sum(weights)) * oofs[-1]
        return log_loss_numpy(labels, blend)
```
에서 `weights`는 `전체 모델의 개수 - 1`개이며, 마지막 가중치는 `(1 - np.sum(weights))`가 되는 형태로 **`constraint 2`** 를 해결했습니다. 이것이 위에서 `get_best_weights` 함수가 가중치를 2개만 리턴한 이유입니다.

그렇지만 이 방법은 **`constraint 1`** 을 해결하지 못했습니다. MoA 4등 솔루션은 `get_best_weights`가 리턴한 각 가중치들이 0과 1 사이의 실수이고 그 합이 1보다 작았지만, 실제로 위와 같이 하면 가중치가 음수가 나오거나 합이 1을 넘어갈 수 있습니다.

references:
1. https://www.kaggle.com/c/lish-moa/discussion/186539#1039438
2. http://www.acme.byu.edu/wp-content/uploads/2016/12/Vol2B-ScipyOptimize-20171.pdf pp.5

In [7]:
get_best_weights(preds, y)

 final_simplex: (array([[0.4993    , 0.49852   , 0.50032001],
       [0.4993    , 0.49852   , 0.50032001],
       [0.4993    , 0.49852   , 0.50032001],
       [0.4993    , 0.49852   , 0.50032001]]), array([0.69314554, 0.69314554, 0.69314554, 0.69314554]))
           fun: 0.6931455441581822
       message: 'Optimization terminated successfully.'
          nfev: 253
           nit: 126
        status: 0
       success: True
             x: array([0.4993    , 0.49852   , 0.50032001])


array([0.4993    , 0.49852   , 0.50032001])

실제로 예시의 결과는 `[0.4993, 0.49852, 0.50032]`으로 그 합이 1이 넘어가며 모델4의 가중치가 음수가 나오게 됨을 볼 수 있습니다.

# 3. Constrained Optimization; SLSQP

  제대로 된 가중치를 찾기 위해서는 Constrained optimization을 하면 된다고 합니다.
  
references:
1. https://www.kaggle.com/tolgadincer/blending-multilabeled-models-with-scipy
2. https://www.kaggle.com/gogo827jz/optimise-blending-weights-with-bonus-0

In [8]:
def func_numpy_metric(weights, oofs, labels):
    oof_blend = np.tensordot(weights, oofs, axes = ((0), (0)))
    return log_loss_numpy(oof_blend, labels)

* Nelder-Mead 예시에서의 `get_score`와 같은 역할을 하는 함수힙니다.

In [9]:
def get_best_weights_constrained(oofs, labels):
    tol = 1e-10
    init_guess = [1 / len(oofs)] * len(oofs)
    bnds = [(0, 1) for _ in range(len(oofs))]
    cons = {'type': 'eq', 
            'fun': lambda x: np.sum(x) - 1, 
            'jac': lambda x: [1] * len(x)}

    res_scipy = minimize(fun = func_numpy_metric, 
                         x0 = init_guess,
                         args=(oofs, labels),
                         method = 'SLSQP',
                         bounds = bnds, 
                         constraints = cons, 
                         tol = tol)

    print(res_scipy)

* Nelder-Mead 방법 대신 SLSQP 라는 방법을 사용하고 있습니다.
<br>

* initial guess의 길이가 oofs의 길이와 같습니다.
<br>

* constraint를 설명하는 `bnds`와 `cons`가 추가 되었습니다.

SLSQP를 이용한 minimize 사용에 대한 자세한 내용은 [링크](https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html#sequential-least-squares-programming-slsqp-algorithm-method-slsqp)를 참고해주시길 바랍니다.

In [10]:
get_best_weights_constrained(preds, y)

     fun: 0.746401813877979
     jac: array([ 0.00000000e+00, -8.51601362e-06, -8.30739737e-06,  2.29398616e-01])
 message: 'Optimization terminated successfully'
    nfev: 35
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([0.39013821, 0.30426809, 0.3055937 , 0.        ])


예시의 결과가 대략 `[0.390, 0.304, 0.306, 0]`으로 그 합이 1이 되며, 모든 가중치가 0과 1사이의 실수임을 확인할 수 있습니다.

---

`SLSQP`는 gradient가 필요한 방법입니다. `minimize`에 `jac`인자를 넘겨주지 않는 경우에는 2-point numerical gradient를 이용해서 gradient를 계산하게 됩니다.

references:
1. https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize
2. https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html#sequential-least-squares-programming-slsqp-algorithm-method-slsqp

`jac`을 정의해서 optimize 할 수도 있습니다.

In [11]:
def grad_func(weights, oofs, labels):
    oof_clip = np.clip(oofs, 1e-15, 1 - 1e-15)
    gradients = np.zeros(oofs.shape[0])
    for i in range(oofs.shape[0]):
        a, b, c = labels, oof_clip[i], np.zeros((oofs.shape[1], oofs.shape[2]))
        for j in range(oofs.shape[0]):
            if j != i:
                c += weights[j] * oof_clip[j]
        gradients[i] = -np.mean((-a*b+(b**2)*weights[i]+b*c)/((b**2)*(weights[i]**2)+2*b*c*weights[i]-b*weights[i]+(c**2)-c))
    return gradients

In [12]:
def get_best_weights_constrained_jac(oofs, labels):
    tol = 1e-10
    init_guess = [1 / preds.shape[0]] * preds.shape[0]
    bnds = [(0, 1) for _ in range(preds.shape[0])]
    cons = {'type': 'eq', 
            'fun': lambda x: np.sum(x) - 1, 
            'jac': lambda x: [1] * len(x)}

    res_scipy = minimize(fun = func_numpy_metric, 
                         x0 = init_guess,
                         args=(oofs, labels),
                         method = 'SLSQP',
                         jac = grad_func,
                         bounds = bnds, 
                         constraints = cons, 
                         tol = tol)

    print('Optimised Weights:', res_scipy)

In [13]:
get_best_weights_constrained_jac(preds, y)

Optimised Weights:      fun: 0.7464018138778526
     jac: array([-2.29415462e-16, -8.52403602e-06, -8.31271598e-06,  2.29398626e-01])
 message: 'Optimization terminated successfully'
    nfev: 7
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([3.90138194e-01, 3.04268099e-01, 3.05593707e-01, 5.33986511e-17])
